### Assignment 16: Intelligent Solving of the 8-Puzzle Problem Using Heuristic Search
Implement and compare heuristic search strategies for solving the 8-Puzzle problem. Specifically, develop solutions using the A* algorithm and Greedy Best-First Search (GBFS), leveraging heuristics like the Manhattan Distance or the number of misplaced tiles.

Problem Setup:

The 8-Puzzle is a sliding puzzle consisting of a 3x3 grid with numbered tiles (1-8) and a blank space. The goal is to arrange the tiles in a specified order (usually numerical) by sliding tiles into the blank space, starting from a given configuration.

Representation of the Problem:
1. Model the puzzle as a 3x3 grid using a 2D list or a flat list of 9 elements.
2. Define the possible moves: Up, Down, Left, Right.
3. Represent the goal state as [1, 2, 3, 4, 5, 6, 7, 8, 0] (where 0 represents the blank space).

Heuristic Functions:
1. Implement heuristic functions to guide the search:
  1. Manhattan Distance: The sum of the absolute differences between the current and target positions of each tile.
  2. Misplaced Tiles: The number of tiles not in their correct positions.

Search Algorithms:
1. A*: Combines the cost to reach a state (g(n)) with the heuristic estimate to the goal h(n).
2. Greedy Best-First Search (GBFS): Considers only the heuristic value (h(n)) for each state.

Performance Evaluation:
1. Compare the performance of A* and GBFS in terms of:
  * Number of nodes expanded.
  * Solution optimality (path cost).
  * Execution time.Implementation Requirements:
2. Write a function to expand possible moves from a given state.
3. Implement a priority queue to explore states based on the chosen heuristic.
4. Ensure the solution handles unsolvable configurations gracefully.
5. Test with multiple initial configurations.

In [1]:
import heapq
import time
from typing import List, Tuple, Optional

# Define the goal state and possible moves.
GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]
MOVE_OFFSETS = {
    'Up': -3,
    'Down': 3,
    'Left': -1,
    'Right': 1
}

def is_solvable(state: List[int]) -> bool:
    """Check if a given 8-puzzle state is solvable using the inversion count."""
    inv_count = 0
    tiles = [tile for tile in state if tile != 0]
    for i in range(len(tiles)):
        for j in range(i + 1, len(tiles)):
            if tiles[i] > tiles[j]:
                inv_count += 1
    return inv_count % 2 == 0

class Node:
    def __init__(self, state: List[int], parent: Optional['Node'] = None, move: Optional[str] = None,
                 g: int = 0, h: int = 0):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # cost to reach current node
        self.h = h  # heuristic cost estimate
        self.f = g + h  # total cost

    def __lt__(self, other: 'Node'):
        return self.f < other.f

def manhattan_distance(state: List[int], goal: List[int] = GOAL_STATE) -> int:
    """Compute the Manhattan distance heuristic."""
    distance = 0
    for i, tile in enumerate(state):
        if tile != 0:
            goal_index = goal.index(tile)
            x1, y1 = i % 3, i // 3
            x2, y2 = goal_index % 3, goal_index // 3
            distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

def misplaced_tiles(state: List[int], goal: List[int] = GOAL_STATE) -> int:
    """Compute the number of misplaced tiles (excluding the blank)."""
    return sum(1 for i in range(len(state)) if state[i] != 0 and state[i] != goal[i])

def get_neighbors(node: Node, heuristic_func) -> List[Node]:
    """Generate all valid neighbor nodes from the current node."""
    neighbors = []
    blank_index = node.state.index(0)
    row, col = blank_index // 3, blank_index % 3

    for move, offset in MOVE_OFFSETS.items():
        new_blank = blank_index + offset

        if move == 'Left' and col == 0:
            continue
        if move == 'Right' and col == 2:
            continue
        if move == 'Up' and row == 0:
            continue
        if move == 'Down' and row == 2:
            continue

        new_state = node.state.copy()
        new_state[blank_index], new_state[new_blank] = new_state[new_blank], new_state[blank_index]
        new_g = node.g + 1
        new_h = heuristic_func(new_state)
        neighbor = Node(state=new_state, parent=node, move=move, g=new_g, h=new_h)
        neighbors.append(neighbor)
    return neighbors

def reconstruct_path(node: Node) -> Tuple[List[str], List[List[int]]]:
    """Reconstruct the path of moves and states from the start to the given node."""
    moves = []
    states = []
    current = node
    while current.parent is not None:
        moves.append(current.move)
        states.append(current.state)
        current = current.parent
    states.append(current.state)
    moves.reverse()
    states.reverse()
    return moves, states

def astar(initial: List[int], heuristic_func) -> Tuple[Optional[List[str]], int, int, float]:
    """A* algorithm for solving the 8-puzzle problem."""
    start_time = time.time()
    start_node = Node(initial, g=0, h=heuristic_func(initial))
    frontier = []
    heapq.heappush(frontier, start_node)
    explored = set()
    nodes_expanded = 0

    while frontier:
        current_node = heapq.heappop(frontier)
        nodes_expanded += 1

        if current_node.state == GOAL_STATE:
            moves, _ = reconstruct_path(current_node)
            end_time = time.time()
            return moves, current_node.g, nodes_expanded, end_time - start_time

        explored.add(tuple(current_node.state))
        for neighbor in get_neighbors(current_node, heuristic_func):
            if tuple(neighbor.state) in explored:
                continue
            heapq.heappush(frontier, neighbor)

    end_time = time.time()
    return None, 0, nodes_expanded, end_time - start_time

def greedy_best_first_search(initial: List[int], heuristic_func) -> Tuple[Optional[List[str]], int, int, float]:
    """Greedy Best-First Search algorithm for the 8-puzzle problem.
    In GBFS, we consider only the heuristic cost (h), ignoring g."""
    start_time = time.time()
    start_node = Node(initial, g=0, h=heuristic_func(initial))
    start_node.f = start_node.h
    frontier = []
    heapq.heappush(frontier, start_node)
    explored = set()
    nodes_expanded = 0

    while frontier:
        current_node = heapq.heappop(frontier)
        nodes_expanded += 1

        if current_node.state == GOAL_STATE:
            moves, _ = reconstruct_path(current_node)
            end_time = time.time()
            return moves, current_node.g, nodes_expanded, end_time - start_time

        explored.add(tuple(current_node.state))
        for neighbor in get_neighbors(current_node, heuristic_func):
            if tuple(neighbor.state) in explored:
                continue
            neighbor.f = neighbor.h
            heapq.heappush(frontier, neighbor)

    end_time = time.time()
    return None, 0, nodes_expanded, end_time - start_time

def print_results(algorithm: str, moves: Optional[List[str]], cost: int, nodes: int, elapsed: float):
    print(f"--- {algorithm} Results ---")
    if moves is None:
        print("No solution found.")
    else:
        print("Solution moves:", moves)
        print("Path cost:", cost)
    print("Nodes expanded:", nodes)
    print("Elapsed time: {:.4f} seconds".format(elapsed))
    print()

initial_state = [1, 2, 3,
                 4, 0, 6,
                 7, 5, 8]

if not is_solvable(initial_state):
  print("The provided puzzle configuration is unsolvable.")
else:
  print("Initial State:")
  for i in range(0, 9, 3):
      print(initial_state[i:i+3])
  print()
# Using Manhattan Distance
print("Using Manhattan Distance Heuristic:")
moves, cost, nodes, elapsed = astar(initial_state, manhattan_distance)
print_results("A* (Manhattan)", moves, cost, nodes, elapsed)
moves, cost, nodes, elapsed = greedy_best_first_search(initial_state, manhattan_distance)
print_results("GBFS (Manhattan)", moves, cost, nodes, elapsed)

# Using Misplaced Tiles
print("Using Misplaced Tiles Heuristic:")
moves, cost, nodes, elapsed = astar(initial_state, misplaced_tiles)
print_results("A* (Misplaced Tiles)", moves, cost, nodes, elapsed)
moves, cost, nodes, elapsed = greedy_best_first_search(initial_state, misplaced_tiles)
print_results("GBFS (Misplaced Tiles)", moves, cost, nodes, elapsed)

Initial State:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Using Manhattan Distance Heuristic:
--- A* (Manhattan) Results ---
Solution moves: ['Down', 'Right']
Path cost: 2
Nodes expanded: 3
Elapsed time: 0.0001 seconds

--- GBFS (Manhattan) Results ---
Solution moves: ['Down', 'Right']
Path cost: 2
Nodes expanded: 3
Elapsed time: 0.0001 seconds

Using Misplaced Tiles Heuristic:
--- A* (Misplaced Tiles) Results ---
Solution moves: ['Down', 'Right']
Path cost: 2
Nodes expanded: 3
Elapsed time: 0.0001 seconds

--- GBFS (Misplaced Tiles) Results ---
Solution moves: ['Down', 'Right']
Path cost: 2
Nodes expanded: 3
Elapsed time: 0.0001 seconds

